<a href="https://colab.research.google.com/github/hiroky1983/AwesomeProject/blob/master/shogi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from torch.optim import optimizer
from torch.utils import data
from prompt_toolkit import output
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# ハイパーパラメーター
learning_rate = 0.001
batch_size = 64
epoches = 5

# データセット
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
training_data = datasets.MNIST(
    'data' , train=True, download=True, transform=transform)

test_data = datasets.MNIST(
    'data', train=False, transform=transform)

# データローダ
train_dataloader = DataLoader(training_data, batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size)

# デバイス
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

# ニューラルネットワークの定義
class Net(nn.Module) :
  def __init__(self) :
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding= 0)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    output = self.fc2(x)
    return output

model = Net()
model.to(device)

# 損失関数
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epoches):
      model.train()
      for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.to(device), target.to(device)

        output = model(data)
        # 損失計算
        loss = loss_fn(output, target)

        # 誤差逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

            #一定間隔ごとに訓練損失を表示
        if (batch_idx + 1) % 100 == 0:
              print('epoch: {}, steps: {}/{}, train loss: {:.6f}'.format(
                  t + 1,
                  batch_idx + 1, len(train_dataloader),
                  loss.item()
          ))

              # エポックの終わりにテストデータすべてを使用して評価する
              model.eval()
              test_loss = 0
              correct = 0

              with torch.no_grad():
                for data, target in test_dataloader:
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    test_loss += loss_fn(output, target).item()
                    correct += (output.argmax(1) == target).type (torch.float).sum().item()

              print('epoch: {}, test loss: {:.6f}, test accuracy: {:.6f}'.format(
                  t + 1,
                  test_loss / len(train_dataloader),
                  correct / len(test_dataloader.dataset)
              ))



epoch: 1, steps: 100/938, train loss: 2.252206
epoch: 1, test loss: 0.376090, test accuracy: 0.324900
epoch: 1, steps: 200/938, train loss: 2.161418
epoch: 1, test loss: 0.363978, test accuracy: 0.549000
epoch: 1, steps: 300/938, train loss: 2.065451
epoch: 1, test loss: 0.343049, test accuracy: 0.690700
epoch: 1, steps: 400/938, train loss: 1.798045
epoch: 1, test loss: 0.307483, test accuracy: 0.736400
epoch: 1, steps: 500/938, train loss: 1.591956
epoch: 1, test loss: 0.250884, test accuracy: 0.765500
epoch: 1, steps: 600/938, train loss: 1.107213
epoch: 1, test loss: 0.184768, test accuracy: 0.800100
epoch: 1, steps: 700/938, train loss: 0.857840
epoch: 1, test loss: 0.135695, test accuracy: 0.834000
epoch: 1, steps: 800/938, train loss: 0.731157
epoch: 1, test loss: 0.107545, test accuracy: 0.852400
epoch: 1, steps: 900/938, train loss: 0.544438
epoch: 1, test loss: 0.091635, test accuracy: 0.868600
epoch: 2, steps: 100/938, train loss: 0.493974
epoch: 2, test loss: 0.079551, test

KeyboardInterrupt: ignored